In [1]:
from gene import Gene, GeneChain
from genetic import Genetic
from selector import TopNSelector
from util import print_row, print_header
from loader import load_data, load_params
from crosser import RandomGeneCrosser
from mutator import RandomGeneChainMutator
import numpy as np

## Load data

In [2]:
data = load_data()
data.head(5)

,COLUMN1,COLUMN2,COLUMN3,COLUMN4,COLUMN5,COLUMN6,COLUMN7,COLUMN8,COLUMN9,COLUMN10
0,3,21,5,0,1.34,0.09,0.38,0.38,0.16,14.333333
1,4,21,5,0,1.42,0.10,0.40,0.40,0.33,14.333333
2,5,21,5,0,1.42,0.21,0.40,0.40,0.33,6.615385
3,5,21,5,0,1.42,0.69,0.40,0.40,0.33,2.047619
4,5,21,5,0,1.43,0.70,0.40,0.40,0.33,2.047619


In [3]:
params = load_params()
params

,TYPE,MIN,MAX,DESCRIPTION
0,float64,-1.0,1.0,distance between order price and level
1,float64,0.5,2.0,stop loss
2,float64,0.5,2.5,stop out
3,float64,0.5,3.0,take profit
4,float64,-2.0,2.0,bu level
5,float64,0.5,3.0,bu condition level
6,int64,1.0,5.0,max stops
7,int64,300.0,3600.0,expiration(in minuts)
8,int64,0.0,1500.0,min time between orders(in minuts)


## Init base gene chain

In [4]:
print_header(("", "Type", "Min", "Max", "Interval"))
base_gene_chain = GeneChain()

for index, column in data.iteritems():
    dtype = column.dtype
    min_value = round(min(column), 9)
    max_value = round(max(column), 9)
    base_gene_chain.add(Gene(dtype.type, max_value, min_value, -1, True))
    
    print_row((index, dtype.name, min_value, max_value, str(True)))

print_header([""]*5)
for index, row in params.iterrows():
    dtype = np.dtype(row["TYPE"])
    min_value = dtype.type(row["MIN"])
    max_value = dtype.type(row["MAX"])
    base_gene_chain.add(Gene(dtype.type, max_value, min_value, -1))
    
    print_row((index+1, dtype.name, min_value, max_value, str(False)))

|          |      Type|       Min|       Max|  Interval|
+----------+----------+----------+----------+----------|
|   COLUMN1|     int64|         3|        15|      True|
|   COLUMN2|     int64|         2|        43|      True|
|   COLUMN3|     int64|         2|        12|      True|
|   COLUMN4|     int64|         0|         3|      True|
|   COLUMN5|   float64|       0.7|       2.1|      True|
|   COLUMN6|   float64|      0.02|       0.8|      True|
|   COLUMN7|   float64|      0.01|       0.6|      True|
|   COLUMN8|   float64|      0.01|       0.6|      True|
|   COLUMN9|   float64|       0.0|       0.6|      True|
|  COLUMN10|   float64|       1.4|      59.0|      True|
|          |          |          |          |          |
+----------+----------+----------+----------+----------|
|         1|   float64|      -1.0|       1.0|     False|
|         2|   float64|       0.5|       2.0|     False|
|         3|   float64|       0.5|       2.5|     False|
|         4|   float64|       0

## Init genetic algorithm

In [5]:
genetic = Genetic(
    max_generations=100,
    start_population=10,
    base_gene_chain=base_gene_chain,
    crosser=RandomGeneCrosser(),
    selector=TopNSelector(10),
    mutator=RandomGeneChainMutator(10))

## Run 

In [6]:
genetic.run()

Generation: 1 | Fitness: 0 (1s.)
Generation: 2 | Fitness: 0 (1s.)
Generation: 3 | Fitness: 0 (2s.)
Generation: 4 | Fitness: 0 (1s.)
Generation: 5 | Fitness: 0 (1s.)
Generation: 6 | Fitness: 0 (2s.)
Generation: 7 | Fitness: 0 (2s.)
Generation: 8 | Fitness: 0 (1s.)
Generation: 9 | Fitness: 0 (2s.)
Generation: 10 | Fitness: 0 (1s.)
Generation: 11 | Fitness: 0 (1s.)
Generation: 12 | Fitness: 0 (1s.)
Generation: 13 | Fitness: 9.0 (1s.)
Generation: 14 | Fitness: 11.0 (2s.)
Generation: 15 | Fitness: 21.0 (2s.)
Generation: 16 | Fitness: 29.0 (3s.)
Generation: 17 | Fitness: 30.0 (3s.)
Generation: 18 | Fitness: 39.0 (3s.)
Generation: 19 | Fitness: 59.0 (4s.)
Generation: 20 | Fitness: 61.0 (4s.)
Generation: 21 | Fitness: 102.0 (7s.)
Generation: 22 | Fitness: 103.0 (7s.)
Generation: 23 | Fitness: 183.0 (9s.)
Generation: 24 | Fitness: 193.0 (11s.)
Generation: 25 | Fitness: 249.0 (16s.)
Generation: 26 | Fitness: 274.0 (14s.)
Generation: 27 | Fitness: 304.0 (16s.)
Generation: 28 | Fitness: 335.0 (17s

In [7]:
for i, gene in enumerate(genetic.best()):
    print(i, gene)

0 [3, 11]
1 [2, 42]
2 [2, 12]
3 [0, 2]
4 [0.7, 1.9830267314366492]
5 [0.1479593208223362, 0.8]
6 [0.12470656701910782, 0.6]
7 [0.12080439596042303, 0.6]
8 [0.0, 0.4877177879195136]
9 [1.4, 37.13513740995522]
10 -0.0
11 2.0
12 1.3
13 1.7
14 1.9
15 0.5
16 2
17 2864
18 5


In [12]:
print("BEST 10 individuals")
for gene_chains in genetic._individuals:    
    for i, gene in enumerate(gene_chains):
        print(i, gene)
    print("SCORE = ", gene_chains.score)
    print("-----------------------")

BEST 10 individuals
0 [3, 11]
1 [2, 42]
2 [2, 12]
3 [0, 2]
4 [0.7, 1.9830267314366492]
5 [0.1479593208223362, 0.8]
6 [0.12470656701910782, 0.6]
7 [0.12080439596042303, 0.6]
8 [0.0, 0.4877177879195136]
9 [1.4, 37.13513740995522]
10 -0.0
11 2.0
12 1.3
13 1.7
14 1.9
15 0.5
16 2
17 2864
18 5
SCORE =  1887.300000000071
-----------------------
0 [3, 11]
1 [2, 42]
2 [2, 12]
3 [0, 2]
4 [0.7, 1.9830267314366492]
5 [0.1479593208223362, 0.8]
6 [0.12470656701910782, 0.6]
7 [0.12080439596042303, 0.6]
8 [0.0, 0.4877177879195136]
9 [1.4, 37.13513740995522]
10 -0.0
11 2.0
12 1.3
13 1.7
14 1.9
15 0.5
16 2
17 2864
18 5
SCORE =  1887.300000000071
-----------------------
0 [3, 11]
1 [2, 42]
2 [2, 12]
3 [0, 2]
4 [0.7, 1.9830267314366492]
5 [0.1479593208223362, 0.8]
6 [0.12470656701910782, 0.6]
7 [0.12080439596042303, 0.6]
8 [0.0, 0.4877177879195136]
9 [1.4, 37.13513740995522]
10 -0.0
11 2.0
12 1.3
13 1.7
14 1.9
15 0.5
16 2
17 1185
18 5
SCORE =  1887.300000000071
-----------------------
0 [3, 11]
1 [2, 42]
